In [ ]:
%pylab inline
plt.style.use("default")

import os 
import sys
import helpers as h
import utils as u
from glob import glob

from scipy.io import loadmat
from scipy.stats import skew, kurtosis

plt.rcParams['figure.figsize'] = (15, 10) 

In [ ]:
data_root = "/Users/duuta/ppp/data/stringer/live_data/"
data_files = [fname.split("/")[-1] for fname in glob(f"{data_root}natimg2800_M*.mat")]



In [ ]:
def clean_unpack(data0):
    """returns clean and zscored signal"""
    resp0, spon0, istim0 = h.unbox(data0)

    resp0 = h.denoise_resp(resp0, spon0)

    resp0 = h.dupSignal(resp0, istim0)

    return resp0, istim0


In [ ]:
_, _, istim = h.unbox(data)

In [ ]:
(istim == 0).nonzero()[0]

In [ ]:
data = loadmat( os.path.join(data_root, data_files[0]), simplify_cells=True)

In [ ]:
resp, istim0= clean_unpack(data)


In [ ]:
np.nonzero(istim0 == 250)

In [ ]:
resp.shape

In [ ]:
data['stim']['resp'].shape

* need to figure out dupSignal function, seems repeats responses are sought and averaged
* need to figure out what happens... NB: in stringer experiment repeat stimulus in alexnet, there are no repeats. 
* these are important to create a 2* N * M stimulus and then compute cvPCA.
* I could pretend this is spontaneous activity and split the signal then procced

# assuming the responses are spontaneous ( in the sense that there are no repeats)


In [ ]:
x, y, z = data['med'].T


In [ ]:
def points_above_line(a, b, c, point_list):
    above_indices, below_indices = [], []
    
    for i, point in enumerate(zip(point_list[0], point_list[1])):
        x, y = point
        if a*x + b*y + c > 0:
            above_indices.append(i)
        else:
            below_indices.append(i)


    return above_indices, below_indices



In [ ]:
point_list = [x, y]


In [ ]:
a, b, c = 59/70, 1, -1168.571

In [ ]:
above_points, below_points = points_above_line(a, b, c, point_list)

In [ ]:
nab = resp[:, :, above_points]
nbl = resp[:, :, below_points]


In [ ]:
def compute_cvPCA(resp0):
    ss0 = u.shuff_cvPCA(resp0, nshuff=10, maxcols=2000)
    ss0 = ss0.mean(axis=0)
    ss0 = ss0 / ss0.sum()

    return ss0


In [ ]:
nbl.shape

In [ ]:
ss0 = compute_cvPCA(nab)

In [ ]:
ss1 = compute_cvPCA(nbl)

In [ ]:
aalpha, _ = u.get_powerlaw(ss0, np.arange(11, 200).astype('int') )
balpha, _ = u.get_powerlaw(ss1, np.arange(11, 200).astype('int'))


In [ ]:
aalpha

In [ ]:
balpha